 

# <p style="color:Orange;">Semantic Chunking</p>  +  <p style="color:Red;">Raptors</p>  for RAG

In [1]:

# %%capture
# !pip install llama-index-packs-raptor

To read about Raptors - https://github.com/run-llama/llama_index/tree/main/llama-index-packs/llama-index-packs-raptor/llama_index/packs/raptor

## Imports

In [2]:
import shutup
shutup.please()

In [3]:
#Imports
from llama_index.llms.openai import OpenAI
import json
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
    Document,
    ServiceContext,
)
from llama_index.core import Settings
#semantic chunking
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)

In [4]:
#using llama-parse for parsing complex documents

To use llama-parse refer to - https://cloud.llamaindex.ai/landing

In [5]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os
# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-"

# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = "sk-"

In [6]:
embed_model=OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-3.5-turbo-0125")

Settings.llm = llm
Settings.embed_model = embed_model

In [7]:
from llama_parse import LlamaParse

documents = LlamaParse(result_type="markdown").load_data('/Users/sahibpreetsingh/Downloads/iPhone.pdf')

Started parsing the file under job_id 678d0d89-482b-4088-8233-e01bd2ae07c9


In [8]:
text_from_pdf = json.loads(documents[0].to_json())['text']

## Defining service context
To reduce the chances of hallucination and set a persona for the CHAT ENGINE

In [9]:
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.2, system_prompt="""
                                                                    You are an expert Analyser and Professor and your job is to answer 
                                                                    questions. Assume that all questions are related to the pdf user provided.
                                                                    Keep your answers technical and based on facts – do not hallucinate features.
                                                                    """))

## Semantic Chunking and Index creation

## To understand semantic chunking :-
1. https://docs.llamaindex.ai/en/latest/examples/node_parsers/semantic_chunking/

In [10]:
embed_model = OpenAIEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=100, embed_model=embed_model
)

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(documents)

index = VectorStoreIndex(nodes,service_context=service_context)

In [11]:
query_engine = index.as_chat_engine(chat_mode="react", verbose=True)
response = query_engine.query("when was the first iphone realeased and who lauched it")

Added user message to memory: when was the first iphone realeased and who lauched it
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "When was the first iPhone released?"}
Got output: The first iPhone was released on June 29, 2007.

=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "Who launched the first iPhone?"}
Got output: CEO Steve Jobs launched the first iPhone.



In [12]:
response.response

'The first iPhone was released on June 29, 2007, and it was launched by CEO Steve Jobs.'

In [13]:
response = query_engine.query("and what was the initial price of 4GB iphone")
response.response

Added user message to memory: and what was the initial price of 4GB iphone
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"What was the initial price of 4GB iPhone?"}
Got output: The initial price of the 4GB model of the iPhone was $499.



'The initial price of the 4GB iPhone was $499.'

In [14]:
response = query_engine.query("and who conducted a study to show that people can wait to buy their wireless phone just to buy iphone?")

Added user message to memory: and who conducted a study to show that people can wait to buy their wireless phone just to buy iphone?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"Who conducted a study to show that people can wait to buy their wireless phone just to buy an iPhone?"}
Got output: Sharma and Wingfield conducted a study to show that people can wait to buy their wireless phone just to buy an iPhone.



In [15]:
response = query_engine.query("by how much apple reduced the iphone price after 2 months of sale?",)
response.response

Added user message to memory: by how much apple reduced the iphone price after 2 months of sale?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"Calculate the percentage decrease in the iPhone price after 2 months of sale."}
Got output: The iPhone price decreased by 33% ($200) after being on the market for only two months.



'Apple reduced the iPhone price by 33% ($200) after 2 months of sale.'

## Raptor Retriever

In [16]:
from llama_index.packs.raptor import RaptorPack

In [17]:
from llama_index.packs.raptor import RaptorRetriever

retriever = RaptorRetriever(
    [],
    embed_model=OpenAIEmbedding(
        model="text-embedding-3-small"
    ),  # used for embedding clusters
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),  # used for generating summaries

    existing_index = index,
    similarity_top_k=2,  # top k for each layer, or overall top-k for collapsed
    mode="collapsed",  # sets default mode
)

In [18]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever, temperature=0.1,verbose=True)


In [19]:
response = query_engine.query("when was the first iphone realeased and who lauched it and also what was place i am asking city name and in that city which specific place ",)


In [20]:
# str(response)
response.response

'The first iPhone was released on June 29, 2007, by Apple Inc. The launch event took place in San Francisco at the Macworld convention.'

In [21]:
response = query_engine.query("by how much apple reduced the iphone price after 2 months of sale? and what percetange is it?",)

In [22]:
response.response

'Apple reduced the iPhone price by $200 after 2 months of sale, which is a 33% reduction from the original price.'